In [1]:
%reload_ext autoreload
%autoreload 2
# BASE
import os
import json
# COOL
from pyeed.core import ProteinRecord

In [2]:
# read in the protein records from the json
current_path = os.path.dirname(os.getcwd())
read_blast_folder = os.path.join(current_path, "TEM-lactamase", "data", "data_blast_search_big_list002_all_10")
# /home/niklas/Desktop/Job_Niklas/TEM-lactamase/data/data_blast_search_big_list001test_all_10
# list all the files in the directory
# read in the saved fetched results
protein_list = []
for file in os.listdir(read_blast_folder):
    with open(os.path.join(read_blast_folder, file), "r") as f:
        protein_list.append(ProteinRecord.from_json(f))

print('The fetched list of results is: ' +  str(len(protein_list)))

# now we check for double entries, we check the ids
count = 0
ids = []
filtered_protein_list = []

for protein in protein_list:
    if protein.id not in ids:
        ids.append(protein.id)
        filtered_protein_list.append(protein)
    else:
        count += 1

print('The number of double entries is: ' + str(count))
print('The number of filtered entries is: ' + str(len(filtered_protein_list)))

The fetched list of results is: 621
The number of double entries is: 544
The number of filtered entries is: 77


In [3]:
# read in the file from json
tem_dic = {}

with open(os.path.join(current_path, "TEM-lactamase", "TEM_Ids.json"), "r") as f:
    tem_dic = json.load(f)

# now read in the ids with the tem names and rewrite the list in a dic
filtered_protein_dic = {}

for protein in filtered_protein_list:
    print(protein.id)
    print(protein.id in tem_dic.keys())
    if protein.id in tem_dic.keys():
        filtered_protein_dic[tem_dic[protein.id]] = protein
    else:
        print('The protein with the id: ' + protein.id + ' is not in the tem_dic')

print('The number of proteins in the filtered protein dic is: ' + str(len(filtered_protein_dic)))
print(filtered_protein_dic)


P80545
False
The protein with the id: P80545 is not in the tem_dic
Q01166
False
The protein with the id: Q01166 is not in the tem_dic
P23954
False
The protein with the id: P23954 is not in the tem_dic
P0A9Z7
False
The protein with the id: P0A9Z7 is not in the tem_dic
P10509
False
The protein with the id: P10509 is not in the tem_dic
Q03170
False
The protein with the id: Q03170 is not in the tem_dic
P37323
False
The protein with the id: P37323 is not in the tem_dic
P16897
False
The protein with the id: P16897 is not in the tem_dic
P35392
False
The protein with the id: P35392 is not in the tem_dic
Q9AHN9
False
The protein with the id: Q9AHN9 is not in the tem_dic
Q06650
False
The protein with the id: Q06650 is not in the tem_dic
Q03680
False
The protein with the id: Q03680 is not in the tem_dic
O08337
False
The protein with the id: O08337 is not in the tem_dic
Q9S424
False
The protein with the id: Q9S424 is not in the tem_dic
O69395
False
The protein with the id: O69395 is not in the tem

In [4]:
# now we run clustal omega on the proteins
from pyeed.tools.clustalo import ClustalOmega

sequences = [protein.sequence for protein in filtered_protein_dic.values()]
sequences = [protein.sequence for protein in filtered_protein_list]

clustalo = ClustalOmega()
alignment = clustalo.align(sequences)
print(alignment)

http://clustalo:5001/align
-3
[Errno -3] Temporary failure in name resolution


ConnectError: PyEED Docker Service is not running.